# Data Cleaning and Feature Engineering Notebook

## Assignment Task 4: Data Cleaning & Feature Engineering

This notebook covers comprehensive data cleaning and feature engineering including:
- Handling missing values
- Removing duplicates
- Encoding categorical variables
- Scaling numerical variables
- Outlier detection and treatment
- Feature creation and transformation
- Data visualization of cleaning results

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import sys
import os

# Add src directory to path
sys.path.append('../src')

# Import our custom modules
from data.data_loader import DataLoader
from data.data_cleaning import DataCleaner, FeatureEngineer
from visualization.plots import DataVisualizer

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Load and Examine Dataset

In [ ]:
# Initialize data loader
loader = DataLoader('../data/raw')

# Load dataset (replace with your actual dataset)
try:
    # Example: dataset = loader.load_dataset('your_dataset.csv')
    print("Please load your dataset using loader.load_dataset('filename.csv')")
    print("For now, using sample data for demonstration.")
    
    # Sample dataset with various data quality issues
    np.random.seed(42)
    n_samples = 1000
    
    sample_data = {
        'id': range(1, n_samples + 1),
        'age': np.random.normal(35, 10, n_samples),
        'income': np.random.lognormal(10, 1, n_samples),
        'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD', None], n_samples, 
                                    p=[0.3, 0.4, 0.2, 0.05, 0.05]),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_samples),
        'experience': np.random.normal(8, 5, n_samples),
        'satisfaction': np.random.uniform(1, 10, n_samples),
        'department': np.random.choice(['Sales', 'Marketing', 'Engineering', 'HR', 'Finance'], n_samples),
        'target': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    }
    
    # Add some missing values
    missing_indices = np.random.choice(n_samples, 100, replace=False)
    for idx in missing_indices[:50]:
        sample_data['income'][idx] = np.nan
    for idx in missing_indices[50:80]:
        sample_data['age'][idx] = np.nan
    for idx in missing_indices[80:]:
        sample_data['satisfaction'][idx] = np.nan
    
    # Add some duplicate rows
    duplicate_indices = np.random.choice(n_samples, 20, replace=False)
    for idx in duplicate_indices:
        sample_data['id'][idx] = np.random.choice(sample_data['id'][:idx] if idx > 0 else [1])
    
    # Add some outliers
    outlier_indices = np.random.choice(n_samples, 10, replace=False)
    for idx in outlier_indices[:5]:
        sample_data['income'][idx] = sample_data['income'][idx] * 10  # Extreme high values
    for idx in outlier_indices[5:]:
        sample_data['age'][idx] = -10  # Invalid negative age
    
    dataset = pd.DataFrame(sample_data)
    loader.dataset = dataset
    loader._extract_dataset_info()
    
    print(f"Dataset loaded with {len(dataset)} rows and {len(dataset.columns)} columns")
    print(f"Initial missing values: {dataset.isnull().sum().sum()}")
    print(f"Initial duplicates: {dataset.duplicated().sum()}")
    
except Exception as e:
    print(f"Error loading dataset: {e}")

## 2. Initial Data Quality Assessment

In [ ]:
print("=" * 50)
print("INITIAL DATA QUALITY ASSESSMENT")
print("=" * 50)

# Dataset summary
dataset_summary = loader.get_dataset_summary()
print(f"Dataset Size: {dataset_summary['dataset_size']}")
print(f"Memory Usage: {dataset_summary['memory_usage']}")
print(f"Numeric Variables: {dataset_summary['numeric_variables']}")
print(f"Categorical Variables: {dataset_summary['categorical_variables']}")
print(f"Total Missing Values: {dataset_summary['total_missing_values']}")

# Missing data analysis
print("\nMISSING DATA ANALYSIS:")
print("-" * 25)
missing_data = pd.DataFrame({
    'Missing Count': loader.dataset.isnull().sum(),
    'Missing Percentage': (loader.dataset.isnull().sum() / len(loader.dataset)) * 100
})
missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values('Missing Percentage', ascending=False)
if missing_data.empty:
    print("No missing data found.")
else:
    print(missing_data)

# Duplicate analysis
print(f"\nDUPLICATE ROWS: {loader.dataset.duplicated().sum()}")

# Data types
print("\nDATA TYPES:")
print("-" * 12)
print(loader.dataset.dtypes)

## 3. Initialize Data Cleaning Components

In [ ]:
# Initialize data cleaner and feature engineer
cleaner = DataCleaner(loader.dataset)
engineer = FeatureEngineer(loader.dataset)
visualizer = DataVisualizer(loader.dataset)

print("Data cleaning components initialized successfully.")
print(f"Dataset shape: {loader.dataset.shape}")

## 4. Handle Missing Values

In [ ]:
print("=" * 40)
print("HANDLING MISSING VALUES")
print("=" * 40)

# Handle missing values using auto strategy
print("\n1. APPLYING AUTO MISSING VALUE STRATEGY")
print("-" * 40)
cleaner.handle_missing_values(strategy='auto')

# Show cleaning log
cleaning_report = cleaner.get_cleaning_report()
print("Cleaning operations performed:")
print(cleaning_report)

# Visualize missing data before and after
print("\n2. MISSING DATA VISUALIZATION")
print("-" * 30)

# Before cleaning
missing_before = loader.dataset.isnull().sum()
missing_after = cleaner.dataset.isnull().sum()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

missing_before[missing_before > 0].plot(kind='bar', ax=ax1, color='red', alpha=0.7)
ax1.set_title('Missing Values Before Cleaning')
ax1.set_xlabel('Columns')
ax1.set_ylabel('Missing Count')
ax1.tick_params(axis='x', rotation=45)

missing_after[missing_after > 0].plot(kind='bar', ax=ax2, color='green', alpha=0.7)
ax2.set_title('Missing Values After Cleaning')
ax2.set_xlabel('Columns')
ax2.set_ylabel('Missing Count')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Missing values reduced from {loader.dataset.isnull().sum().sum()} to {cleaner.dataset.isnull().sum().sum()}")

## 5. Remove Duplicates

In [ ]:
print("=" * 30)
print("REMOVING DUPLICATES")
print("=" * 30)

# Remove duplicates
print(f"\nDuplicate rows before cleaning: {cleaner.dataset.duplicated().sum()}")
cleaner.remove_duplicates()
print(f"Duplicate rows after cleaning: {cleaner.dataset.duplicated().sum()}")

# Update cleaning log
cleaning_report = cleaner.get_cleaning_report()
print("\nUpdated cleaning operations:")
print(cleaning_report)

## 6. Handle Outliers

In [ ]:
print("=" * 25)
print("HANDLING OUTLIERS")
print("=" * 25)

# Detect outliers before removal
numeric_columns = cleaner.dataset.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNUMERICAL COLUMNS: {numeric_columns}")

# Visualize outliers using boxplots
print("\n1. OUTLIER VISUALIZATION BEFORE CLEANING")
print("-" * 42)
if len(numeric_columns) > 0:
    n_cols = min(4, len(numeric_columns))
    n_rows = (len(numeric_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    elif n_rows == 1 or n_cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(numeric_columns):
        if i < len(axes):
            cleaner.dataset.boxplot(column=col, ax=axes[i])
            axes[i].set_title(f'{col} - Before Outlier Removal')
    
    # Hide empty subplots
    for i in range(len(numeric_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Remove outliers using IQR method
print("\n2. REMOVING OUTLIERS USING IQR METHOD")
print("-" * 38)
initial_rows = len(cleaner.dataset)
cleaner.remove_outliers(method='iqr', threshold=1.5)
final_rows = len(cleaner.dataset)
removed_outliers = initial_rows - final_rows

print(f"Rows before outlier removal: {initial_rows}")
print(f"Rows after outlier removal: {final_rows}")
print(f"Outliers removed: {removed_outliers}")

# Visualize outliers after removal
print("\n3. OUTLIER VISUALIZATION AFTER CLEANING")
print("-" * 39)
if len(numeric_columns) > 0:
    n_cols = min(4, len(numeric_columns))
    n_rows = (len(numeric_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    elif n_rows == 1 or n_cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(numeric_columns):
        if i < len(axes) and col in cleaner.dataset.columns:
            cleaner.dataset.boxplot(column=col, ax=axes[i])
            axes[i].set_title(f'{col} - After Outlier Removal')
    
    # Hide empty subplots
    for i in range(len(numeric_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Update cleaning log
cleaning_report = cleaner.get_cleaning_report()
print("\nUpdated cleaning operations:")
print(cleaning_report)

## 7. Encode Categorical Variables

In [ ]:
print("=" * 40)
print("ENCODING CATEGORICAL VARIABLES")
print("=" * 40)

# Identify categorical columns
categorical_columns = cleaner.dataset.select_dtypes(include=['object']).columns.tolist()
print(f"\nCATEGORICAL COLUMNS: {categorical_columns}")

# Show unique values for each categorical column
print("\nUNIQUE VALUES IN CATEGORICAL COLUMNS:")
print("-" * 40)
for col in categorical_columns:
    unique_vals = cleaner.dataset[col].nunique()
    print(f"{col}: {unique_vals} unique values")
    if unique_vals <= 10:  # Show values for low cardinality
        print(f"  Values: {list(cleaner.dataset[col].unique())}")

# Apply label encoding
print("\nAPPLYING LABEL ENCODING")
print("-" * 25)
cleaner.encode_categorical_variables(method='label')

# Show data types after encoding
print("\nDATA TYPES AFTER ENCODING:")
print("-" * 28)
print(cleaner.dataset.dtypes)

# Update cleaning log
cleaning_report = cleaner.get_cleaning_report()
print("\nUpdated cleaning operations:")
print(cleaning_report)

## 8. Scale Numerical Variables

In [ ]:
print("=" * 35)
print("SCALING NUMERICAL VARIABLES")
print("=" * 35)

# Identify numerical columns
numerical_columns = cleaner.dataset.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNUMERICAL COLUMNS: {numerical_columns}")

# Show statistics before scaling
print("\nSTATISTICS BEFORE SCALING:")
print("-" * 28)
print(cleaner.dataset[numerical_columns].describe())

# Apply standard scaling
print("\nAPPLYING STANDARD SCALING")
print("-" * 25)
cleaner.scale_numerical_variables(method='standard')

# Show statistics after scaling
print("\nSTATISTICS AFTER SCALING:")
print("-" * 27)
print(cleaner.dataset[numerical_columns].describe())

# Visualize scaling effect
print("\nVISUALIZING SCALING EFFECT")
print("-" * 27)
if len(numerical_columns) > 0:
    n_cols = min(4, len(numerical_columns))
    n_rows = (len(numerical_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    elif n_rows == 1 or n_cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(numerical_columns):
        if i < len(axes):
            cleaner.dataset[col].hist(bins=30, ax=axes[i], alpha=0.7, color='skyblue')
            axes[i].set_title(f'{col} - After Scaling')
            axes[i].set_xlabel('Value')
            axes[i].set_ylabel('Frequency')
    
    # Hide empty subplots
    for i in range(len(numerical_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Update cleaning log
cleaning_report = cleaner.get_cleaning_report()
print("\nUpdated cleaning operations:")
print(cleaning_report)

## 9. Feature Engineering

In [ ]:
print("=" * 30)
print("FEATURE ENGINEERING")
print("=" * 30)

# Initialize feature engineer with cleaned data
engineer = FeatureEngineer(cleaner.dataset)

# Identify numerical columns for feature engineering
numerical_columns = engineer.dataset.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNUMERICAL COLUMNS FOR FEATURE ENGINEERING: {numerical_columns}")

# Create interaction features (example)
print("\n1. CREATING INTERACTION FEATURES")
print("-" * 33)
if len(numerical_columns) >= 2:
    # Create interaction between first two numerical columns
    column_pairs = [(numerical_columns[0], numerical_columns[1])]
    engineer.create_interaction_features(column_pairs)
    print(f"Created interaction feature: {numerical_columns[0]}_x_{numerical_columns[1]}")

# Create binned features (example)
print("\n2. CREATING BINNED FEATURES")
print("-" * 27)
if len(numerical_columns) > 0:
    # Create binned feature for the first numerical column
    engineer.create_binned_features([numerical_columns[0]], bins=5)
    print(f"Created binned feature: {numerical_columns[0]}_binned")

# Show feature engineering report
feature_report = engineer.get_feature_report()
print("\nFeature engineering operations:")
print(feature_report)

# Show final dataset info
print("\nFINAL DATASET INFORMATION:")
print("-" * 26)
print(f"Shape: {engineer.dataset.shape}")
print(f"Columns: {list(engineer.dataset.columns)}")
print(f"Data types:\n{engineer.dataset.dtypes}")

## 10. Save Cleaned and Engineered Data

In [ ]:
print("=" * 40)
print("SAVING PROCESSED DATA")
print("=" * 40)

# Save cleaned and engineered dataset
try:
    # Create processed data directory if it doesn't exist
    os.makedirs('../data/processed', exist_ok=True)
    
    # Save the final processed dataset
    engineer.dataset.to_csv('../data/processed/cleaned_dataset.csv', index=False)
    print("✓ Cleaned dataset saved to data/processed/cleaned_dataset.csv")
    
    # Save cleaning report
    with open('../reports/cleaning_report.txt', 'w') as f:
        f.write("DATA CLEANING REPORT\n")
        f.write("=" * 50 + "\n\n")
        f.write(cleaner.get_cleaning_report())
        f.write("\n\nFEATURE ENGINEERING REPORT\n")
        f.write("=" * 50 + "\n\n")
        f.write(engineer.get_feature_report())
    
    print("✓ Cleaning and feature engineering reports saved to reports/cleaning_report.txt")
    
except Exception as e:
    print(f"Error saving data: {e}")

# Show final summary
print("\nFINAL SUMMARY:")
print("-" * 15)
print(f"Original dataset shape: {loader.dataset.shape}")
print(f"Final dataset shape: {engineer.dataset.shape}")
print(f"Rows removed: {loader.dataset.shape[0] - engineer.dataset.shape[0]}")
print(f"Columns added: {engineer.dataset.shape[1] - loader.dataset.shape[1]}")
print(f"Missing values in final dataset: {engineer.dataset.isnull().sum().sum()}")
print(f"Duplicate rows in final dataset: {engineer.dataset.duplicated().sum()}")

## 11. Key Findings and Recommendations

In [ ]:
print("=" * 45)
print("KEY FINDINGS AND RECOMMENDATIONS")
print("=" * 45)

print("""
KEY FINDINGS:
-------------
1. Data Quality Issues Addressed:
   - Missing values: Successfully handled using appropriate strategies
   - Duplicate records: Removed to ensure data integrity
   - Outliers: Identified and removed using IQR method
   - Inconsistent data types: Standardized through encoding

2. Data Transformation Results:
   - Categorical variables encoded for machine learning compatibility
   - Numerical variables scaled for better model performance
   - New features created through engineering techniques
   - Dataset standardized for consistent analysis

3. Dataset Improvements:
   - Improved data quality and consistency
   - Enhanced feature set for better model performance
   - Ready for machine learning algorithms
   - Properly documented cleaning process

RECOMMENDATIONS:
----------------
1. For Production Use:
   - Implement more sophisticated outlier detection methods
   - Consider domain-specific imputation strategies
   - Validate engineered features with domain experts
   - Implement automated data quality monitoring

2. For Model Development:
   - Use the cleaned dataset for model training
   - Consider feature selection techniques
   - Validate scaling methods with cross-validation
   - Document all preprocessing steps for reproducibility

3. For Future Improvements:
   - Implement more advanced feature engineering techniques
   - Add data validation and quality checks
   - Consider automated hyperparameter tuning for scaling
   - Implement data versioning and lineage tracking
""")

print("\nDATA CLEANING AND FEATURE ENGINEERING COMPLETED SUCCESSFULLY!")